In [ ]:
import matplotlib.pyplot as plt
!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb libav-tools xorg-dev python-opengl libboost-all-dev libsdl2-dev swig

!pip3 install gym[atari]



In [1]:
import tensorflow as tf
import numpy as np
import gym
import tensorflow.contrib.slim as slim

/home/adithya/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
batch_size=10
discount_factor=0.9
alpha=0.9
replay_memory_size=10000

In [3]:
in_width=210
in_height=160
input_channels=3
output_shape=2
truncated_backprop_length = 10


In [4]:
#Placeholders

sess=tf.InteractiveSession()
input_tensor=tf.placeholder(tf.float32,shape=(None,in_width,in_height,input_channels),name="placeholder_x")
labels=tf.placeholder(tf.float32,shape=(None,2),name="placeholder_y")
hidden_state=tf.placeholder(tf.float32,shape=(1,100),name="placeholder_hidden")
cell_state=tf.placeholder(tf.float32,shape=(1,100),name="placeholder_cell")
q_val=tf.placeholder(tf.float32,shape=(1),name="placeholder_q")
train_length = tf.placeholder(dtype=tf.int32)


In [5]:



bias_conv_1 = tf.get_variable(name='bias_conv_1',shape=[6], initializer=tf.constant_initializer(0.0))
bias_conv_2 = tf.get_variable(name='bias_conv_2',shape=[12], initializer=tf.constant_initializer(0.0))
bias_conv_3 = tf.get_variable(name='bias_conv_3',shape=[24], initializer=tf.constant_initializer(0.0))



In [6]:

def create_conv_layer(name,n_filters,filter_width,filter_height,filter_depth):
    conv_weights=tf.get_variable(name=name,
        shape=[filter_height,filter_width,filter_depth,n_filters],
        dtype="float32",
        initializer=tf.contrib.layers.xavier_initializer())
    return conv_weights




conv_1_weights=create_conv_layer("conv1",6,3,3,3)
conv_2_weights=create_conv_layer("conv2",12,3,3,6)
conv_3_weights=create_conv_layer("conv3",24,3,3,12)
    

In [7]:
conv_4_weights=create_conv_layer("conv4",3,1,1,24)


In [8]:
fc1=tf.get_variable(name='action_fc1',shape=[2,100],
                   dtype="float32",
        initializer=tf.contrib.layers.xavier_initializer())
b1=tf.get_variable(name='action_bias',shape=[100],initializer=tf.constant_initializer(0.0))

In [9]:





w2=tf.get_variable(name='output__weights',shape=[100,2],
                   dtype="float32",
        initializer=tf.contrib.layers.xavier_initializer())
b2=tf.get_variable(name='output',shape=[2],initializer=tf.constant_initializer(0.0))




In [10]:
w3=tf.get_variable(name='w3',shape=[370,100],
                   dtype="float32",
        initializer=tf.contrib.layers.xavier_initializer())
b3=tf.get_variable(name='b3',shape=[100],initializer=tf.constant_initializer(0.0))

In [11]:
in_training_mode = tf.placeholder(tf.bool)


conv1=tf.nn.relu(tf.nn.conv2d(input_tensor,conv_1_weights,strides=[1,1,1,1],padding='SAME')+bias_conv_1)
pool1=tf.nn.max_pool(conv1,strides=[1,2,2,1],ksize=[1,2,2,1],padding='SAME')
bn1=tf.layers.batch_normalization(pool1,training=in_training_mode)

conv2=tf.nn.relu(tf.nn.conv2d(bn1,conv_2_weights,strides=[1,1,1,1],padding='SAME')+bias_conv_2)
pool2=tf.nn.max_pool(conv2,strides=[1,2,2,1],ksize=[1,2,2,1],padding='SAME')
bn2=tf.layers.batch_normalization(pool2,training=in_training_mode)

conv3=tf.nn.relu(tf.nn.conv2d(bn2,conv_3_weights,strides=[1,1,1,1],padding='SAME')+bias_conv_3)
pool3=tf.nn.max_pool(conv3,strides=[1,3,2,1],ksize=[1,3,2,1],padding='SAME')
bn3=tf.layers.batch_normalization(pool3,training=in_training_mode)

conv4=tf.nn.relu(tf.nn.conv2d(bn3,conv_4_weights,strides=[1,1,1,1],padding='SAME'))
pool4=tf.nn.max_pool(conv4,strides=[1,2,2,1],ksize=[1,2,2,1],padding='SAME')
bn4=tf.layers.batch_normalization(pool4,training=in_training_mode)



In [12]:

fc_1=tf.nn.tanh(tf.add(tf.matmul(labels,fc1),b1))


In [13]:
tf.__version__


'1.8.0'

In [14]:
reshaped=tf.reshape(bn4,[-1,9*10*3])

In [15]:
concatenate=tf.concat([reshaped,fc_1],axis=-1)
concatenate

<tf.Tensor 'concat:0' shape=(?, 370) dtype=float32>

In [16]:

fc_2=tf.add(tf.matmul(concatenate,w3),b3)

In [17]:
flattened=slim.flatten(fc_2)
convFlat=tf.nn.tanh(tf.reshape(flattened,[1,train_length,100]))
rnn_cell = tf.contrib.rnn.BasicLSTMCell(100)
# seq = tf.split(concatenate,tf.reduce_mean(concatenate,input_length[0])
#             ,axis=0)
# seq
rnn_cell.state_size
print(convFlat)

Tensor("Tanh_1:0", shape=(1, ?, 100), dtype=float32)


In [18]:
outputs, states = tf.nn.dynamic_rnn(rnn_cell, convFlat, dtype=tf.float32,
                                   initial_state=tf.nn.rnn_cell.LSTMStateTuple(hidden_state,cell_state))
outputs

<tf.Tensor 'rnn/transpose_1:0' shape=(1, ?, 100) dtype=float32>

In [19]:
final_output=tf.reshape(outputs[0,-1],[1,100])
logits=tf.add(tf.matmul(final_output,w2),b2)
loss=tf.square(tf.reduce_max(logits)-q_val)
optimizer = tf.train.GradientDescentOptimizer(0.0001)

train=optimizer.minimize(loss)


In [20]:


sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()



In [21]:
states

LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(1, 100) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(1, 100) dtype=float32>)

In [22]:
def softmax(X):
    return 1/(1+np.exp(-X))

In [23]:
def forward_pass(input_tens,action_tens,hidden_state_vec,cell_state_vec):
        probs,hidden=sess.run([logits,states],feed_dict={input_tensor:input_tens,
                                           labels:action_tens,
                                           in_training_mode:False,
                                           hidden_state:hidden_state_vec,
                                            cell_state:cell_state_vec,
                                                         train_length:1
                                               })
        return probs,hidden
    

In [24]:
#create Replay memory

def create_replay_memory(size,epsillon,render=False):
    action_tensor=np.array([0.0,0.0],"float32")
    env=gym.make("Pong-v0")
    prev_observation=env.reset()
    replay_memory=[]
    aggregate=np.array([0.0,0.0])
    hidden_stat=np.zeros((1,100))
    cell_stat=np.zeros((1,100))
    total_reward=0
    for i in range(size):
        if render==True:
            env.render()
        next_action=np.array([0.0,0.0])

        
       

        probs,hidden=forward_pass(np.array([prev_observation]),
                                 np.array([action_tensor]),
                                  hidden_stat,
                                  cell_stat)
        
        if np.random.uniform()<epsillon: 
            observation, reward, done, info=env.step(np.argmax(softmax(probs[0]))+2)
            next_action[np.argmax(softmax(probs[0]))]=1.0            
        else:
            
            if np.random.uniform()<0.5:
                choice=2
            else:
                choice=3
            observation, reward, done, info=env.step(choice)
            
            next_action[choice-2]=1.0
            

            
        total_reward+=reward                                             
        if done==True:
            prev_observation= env.reset() 
            action_tensor=np.array([0.0,0.0],"float32")
            hidden_stat=np.zeros((1,100))
            cell_stat=np.zeros((1,100))


            
        else:    
            replay_memory.append({'a_t_1':action_tensor,
                                  'o_t':prev_observation/255,
                                  'r_t':reward,
                                  'a_t':next_action,
                                  'o_t_1':observation/255,
                                  'h_t':hidden.h,
                                 'c_t':hidden.c,
                                  'h_t_1':hidden_stat,
                                 'c_t_1':cell_stat})
            action_tensor=next_action
            prev_observation=observation
            hidden_stat=hidden.h
            cell_stat=hidden.c
            aggregate+=next_action
            
        
    print(total_reward)
    print(aggregate)
    return replay_memory

In [33]:


memory=create_replay_memory(1000,1,render=True) 

-20.0
[1000.    0.]


In [25]:
def random_sample(array,length):
    index=int(np.random.uniform(10,length-20))
    sample=array[index:index+10]
    action_list=[a['a_t_1'] for a in sample]
    obs_list=[o['o_t'] for o in sample]
    initial_hidden=sample[0]['h_t_1']
    initial_cell=sample[0]['c_t_1']
    return (action_list,obs_list,initial_hidden,initial_cell,sample)

In [26]:
def train_network(iterations,length):
    for i in range(iterations):
        lo_sum=np.array([0.0])
        action_list,obs_list,initial_hidden,initial_cell,sample=random_sample(memory,length)
        final_hidden,final_cell,final_obs,final_action,final_reward=sample[-1]['h_t'],sample[-1]['c_t'],\
                                                                    sample[-1]['o_t'],sample[-1]['a_t'],\
                                                                    sample[-1]['r_t']
            
        #run through target network to get q values
        probs,hidden=forward_pass(np.array([final_obs]),np.array([final_action]),final_cell,final_hidden)
        q_value=final_reward+alpha*np.max(probs)
        
        #training_pass
        lo,_=sess.run([loss,train],feed_dict={input_tensor:np.array(obs_list),
                                           labels:np.array(action_list),
                                           in_training_mode:True,
                                           hidden_state:np.zeros([1,100]),
                                            cell_state:np.zeros([1,100]),
                                                  q_val:np.array([q_value]),
                                              train_length:10
                                               })
        lo_sum+=lo
        if i%100==0:
            print('loss: {}'.format(lo_sum[0]))  

In [33]:
for i in range(1000):
    memory=create_replay_memory(1000,0.95,render=True) 
    print("episode:{}".format(i))
    train_network(100,1000)

    

-18.0
[ 79. 921.]
episode:0
loss: 7.851629197830334e-05
-13.0
[133. 867.]
episode:1
loss: 0.02512509934604168
-18.0
[ 68. 932.]
episode:2


KeyboardInterrupt: 

In [35]:
save_path = saver.save(sess, "tmp/model_100_epochs.ckpt")
print("Model saved in path: %s" % save_path)  
  



Model saved in path: tmp/model_100_epochs.ckpt


In [ ]:
!ls tmp/

In [ ]:
from google.colab import files
!ls tmp/


files.download('tmp/checkpoint')
files.download('tmp/model_100_epochs.ckpt.data-00000-of-00001')
files.download('tmp/model_100_epochs.ckpt.index')
files.download('tmp/model_100_epochs.ckpt.meta')





In [32]:

new_saver = tf.train.import_meta_graph('./new_weights/model_100_epochs.ckpt.meta')


new_saver.restore(sess,tf.train.latest_checkpoint('./new_weights/'))
b_3=sess.run(b3)
print(b_3)
memory=create_replay_memory(1000,1) 

INFO:tensorflow:Restoring parameters from ./new_weights/model_100_epochs.ckpt
[ 2.90165899e-05  1.82738695e-05  5.05229036e-05 -5.81715199e-07
  1.80585594e-05  4.27134955e-06  1.85739500e-05 -8.97728205e-06
 -7.20083608e-06 -2.46338259e-05 -9.15445617e-06  4.27714303e-06
 -2.09049085e-06  1.69393097e-05  6.88303908e-06  1.28516467e-06
 -4.08361739e-05 -9.84408689e-06 -2.61927016e-05  3.38725522e-06
 -1.76129015e-06 -6.53913639e-06  1.45678750e-05  1.17284280e-05
  9.92782316e-06 -1.95280927e-06  9.68210770e-06  5.15530119e-05
  1.69718642e-05  7.32752687e-06 -4.89312988e-05  7.23333869e-06
  2.40035683e-06  2.42049791e-05  2.81521352e-06  1.00904745e-05
 -1.27832463e-05 -2.77402955e-06 -7.32917715e-06  1.77654056e-05
 -2.69641296e-05 -9.77892000e-07 -3.50454793e-05 -4.68125199e-06
 -6.98776830e-06  1.05977470e-05  6.15915633e-06 -1.02855956e-05
 -3.57612180e-07  2.28752942e-05 -3.71702736e-05  1.53016463e-05
 -8.14708255e-06  1.85057252e-05 -1.41846067e-06 -4.49880827e-06
  6.27215059

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))